In [1]:
import corner
import numpy as np
from matplotlib import pyplot as plt
import json
from tqdm import tqdm
import sys
import shutil

sys.path.append("../")

import tdpy

plt.rcParams.update({
  "text.usetex": True,
  "font.family": "Computer Modern Roman",
  "font.size":    8.0
})

In [ ]:
n = 2000
run_systems = np.load("../data/compact_systems_run_composite.npy", allow_pickle=True)

# dirs = ["no_inj", "single_inj", "binary_inj"]
dirs = ["bin_inj"]

systems = [sys["name"] for sys in run_systems]
# systems = ["TOI-1246"]

results = {}
    

for dir in dirs:
    results[dir] = {}
    for system in systems:
        results[dir][system] = {}

        n_act = 0
        none_survived = False
        try:
            with open(f"../output/{dir}/{system}/NONE_SURVIVED.txt") as f:
                n_act = f.readline()
                none_survived = True
        except Exception as e:
            pass

        if not none_survived:
            for i in tqdm(range(1, n)):
                try:
                    summary = np.load(f"../output/{dir}/{system}/{i}/summary.npy")
                    with open(f"../output/{dir}/{system}/{i}/config.json") as f:
                        cfg = json.load(f)
                    elements = np.load(f"../output/{dir}/{system}/{i}/elements.npy")
                    if n_act==0:
                        results[dir][system]["a_i"] = elements[0,:,0]
                        results[dir][system]["a_f"] = elements[-1,:,0]
                        if dir[:3] == "bin":
                            results[dir][system]["bin_m1"] = [cfg["binary"]["m1"]]
                            results[dir][system]["bin_m2"] = [cfg["binary"]["m2"]]
                            results[dir][system]["bin_esys"] = [cfg["binary"]["e_sys"]]
                            results[dir][system]["bin_e"] = [cfg["binary"]["e"]]
                            results[dir][system]["bin_d"] = [cfg["binary"]["d"]]
                            results[dir][system]["bin_a"] = [cfg["binary"]["a"]]

                    else:
                        results[dir][system]["a_i"] = np.vstack(
                            (results[dir][system]["a_i"], elements[0,:,0]))
                        results[dir][system]["a_f"] = np.vstack(
                            (results[dir][system]["a_f"], elements[-1,:,0]))
                        if dir[:3] == "bin":
                            results[dir][system]["bin_m1"].append(cfg["binary"]["m1"])
                            results[dir][system]["bin_m2"].append(cfg["binary"]["m2"])
                            results[dir][system]["bin_esys"].append(cfg["binary"]["e_sys"])
                            results[dir][system]["bin_e"].append(cfg["binary"]["e"])
                            results[dir][system]["bin_d"].append(cfg["binary"]["d"])
                            results[dir][system]["bin_a"].append(cfg["binary"]["a"])

                    n_act += 1
                    a_i, a_f =  results[dir][system]["a_i"],  results[dir][system]["a_f"]
                    if (np.sum(((a_i-a_f)/a_i)[:,1]<0.1) > 0):
                        pass
                except Exception as e:
                    # print(e)
                    pass
            try:
                if dir[:3] == "bin":                
                    results[dir][system]["bin_m1"] = np.array(results[dir][system]["bin_m1"])
                    results[dir][system]["bin_m2"] = np.array(results[dir][system]["bin_m2"])
                    results[dir][system]["bin_esys"] = np.array(results[dir][system]["bin_esys"])
                    results[dir][system]["bin_e"] = np.array(results[dir][system]["bin_e"])
                    results[dir][system]["bin_d"] = np.array(results[dir][system]["bin_d"])
                    results[dir][system]["bin_a"] = np.array(results[dir][system]["bin_a"])
                print(system, n_act)
            except:
                pass
            try:
                a_i, a_f =  results[dir][system]["a_i"],  results[dir][system]["a_f"]
                
                results[dir][system]["n_survived"] = np.sum(((a_i-a_f)/a_i)[:,1]<0.1)
                results[dir][system]["n_act"] = n_act
                results[dir][system]["survival_frac"] = np.sum(((a_i-a_f)/a_i)[:,1]<0.1)/n_act
                print(f"\n\n{system} Survival Fraction: {np.sum(((a_i-a_f)/a_i)[:,1]<0.1)/n_act} for {n_act} systems")
            except:
                print(f"Error processing {system}")
                results[dir][system]["n_survived"] = np.nan
                results[dir][system]["n_act"] = np.nan
                results[dir][system]["survival_frac"] = np.nan
                
            if (results[dir][system]["n_survived"] < 1e-20):
                with open(f"../output/{dir}/{system}/NONE_SURVIVED.txt", "w+") as f:
                    f.write(f"{n_act}")
                print(f"removing ../output/{dir}/{system}/...")
                for i in tqdm(range(1, n_act)):
                    shutil.rmtree(f"../output/{dir}/{system}/{i}")
                
                
                
        else:
                print(f"NONE SURVIVED in {system}")
                results[dir][system]["n_survived"] = 0
                results[dir][system]["n_act"] = n_act
                results[dir][system]["survival_frac"] = 0
                
            
    


100%|██████████| 1999/1999 [00:49<00:00, 40.70it/s]


HD 108236 1000


HD 108236 Survival Fraction: 0.0 for 1000 systems
removing ../output/bin_inj/HD 108236/...


100%|██████████| 1999/1999 [00:32<00:00, 60.82it/s] 


HD 23472 1000


HD 23472 Survival Fraction: 0.0 for 1000 systems
removing ../output/bin_inj/HD 23472/...


100%|██████████| 1999/1999 [00:10<00:00, 194.36it/s]


...

100%|██████████| 1999/1999 [00:23<00:00, 83.45it/s]


Kepler-342 1000


Kepler-342 Survival Fraction: 0.208 for 1000 systems


100%|██████████| 1999/1999 [00:22<00:00, 90.49it/s]


Kepler-37 1000


Kepler-37 Survival Fraction: 0.019 for 1000 systems


 17%|█▋        | 346/1999 [00:04<00:20, 81.28it/s]

In [ ]:
/scratch/gpfs/wcukier/binary_planets/output/no_inj_large_sample/HD 108236/NONE_SURVIVED.txt

In [4]:
systems

['HD 108236',
 'HD 191939',
 'HD 23472',
 'K2-133',
 'K2-138',
 'K2-266',
 'K2-285',
 'K2-32',
 'K2-72',
 'KOI-94',
 'Kepler-102',
 'Kepler-107',
 'Kepler-11',
 'Kepler-150',
 'Kepler-167',
 'Kepler-169',
 'Kepler-172',
 'Kepler-186',
 'Kepler-197',
 'Kepler-20',
 'Kepler-208',
 'Kepler-215',
 'Kepler-220',
 'Kepler-221',
 'Kepler-224',
 'Kepler-235',
 'Kepler-24',
 'Kepler-251',
 'Kepler-256',
 'Kepler-265',
 'Kepler-286',
 'Kepler-292',
 'Kepler-296',
 'Kepler-299',
 'Kepler-306',
 'Kepler-32',
 'Kepler-33',
 'Kepler-341',
 'Kepler-402',
 'Kepler-444',
 'Kepler-62',
 'Kepler-79',
 'Kepler-82',
 'TOI-1246',
 'TOI-178',
 'TOI-561',
 'TOI-700',
 'TRAPPIST-1',
 'V1298 Tau']

In [8]:
n = 10000
run_systems = np.load("../data/compact_systems_run_composite.npy", allow_pickle=True)

# dirs = ["no_inj", "single_inj", "binary_inj"]
dirs = ["bin_inj"]

systems = [sys["name"] for sys in run_systems]
# systems = ["TOI-1246"]

valid_sys = [ "Kepler-324", "Kepler-220", "Kepler-265", "Kepler-306"]
# run_sys = ["Kepler-106",  "Kepler-150",  "Kepler-172",  "Kepler-20",   
#            "Kepler-251",  "Kepler-286",  "Kepler-306",  "Kepler-341",  "Kepler-444",  "Kepler-79",
#             "Kepler-132",  "Kepler-164",  "Kepler-197",  "Kepler-220",  "Kepler-265", 
#             "Kepler-299",  "Kepler-324",  "Kepler-342",  "Kepler-62",   "Kepler-82"]

results = {}
    
good_sys = []

for dir in dirs:
    results[dir] = {}
    for sys in run_systems:
        system=sys["name"]
        results[dir][system] = {}

        n_act = 0
        none_survived = False
        try:
            with open(f"../output/{dir}/{system}/NONE_SURVIVED.txt") as f:
                n_act = f.readline()
                none_survived = True
                if int(n_act) < 1900:
                    print(n_act)
                    # good_sys.append(sys)
        except Exception as e:
            print(e)
            # if system in valid_sys:
            good_sys.append(sys)


[Errno 2] No such file or directory: '../output/bin_inj/HD 108236/NONE_SURVIVED.txt'
[Errno 2] No such file or directory: '../output/bin_inj/HD 110067/NONE_SURVIVED.txt'
[Errno 2] No such file or directory: '../output/bin_inj/HD 23472/NONE_SURVIVED.txt'
[Errno 2] No such file or directory: '../output/bin_inj/Kepler-324/NONE_SURVIVED.txt'
[Errno 2] No such file or directory: '../output/bin_inj/KOI-351/NONE_SURVIVED.txt'
[Errno 2] No such file or directory: '../output/bin_inj/Kepler-102/NONE_SURVIVED.txt'
[Errno 2] No such file or directory: '../output/bin_inj/Kepler-106/NONE_SURVIVED.txt'
[Errno 2] No such file or directory: '../output/bin_inj/Kepler-107/NONE_SURVIVED.txt'
[Errno 2] No such file or directory: '../output/bin_inj/Kepler-11/NONE_SURVIVED.txt'
[Errno 2] No such file or directory: '../output/bin_inj/Kepler-122/NONE_SURVIVED.txt'
[Errno 2] No such file or directory: '../output/bin_inj/Kepler-132/NONE_SURVIVED.txt'
[Errno 2] No such file or directory: '../output/bin_inj/Kepler

In [9]:
len(good_sys)

54

In [10]:
np.save("../data/compact_systems_run_composite_good", good_sys)


In [14]:
np.sort([sys["name"] for sys in good_sys])

array(['Kepler-106', 'Kepler-132', 'Kepler-150', 'Kepler-164',
       'Kepler-167', 'Kepler-169', 'Kepler-172', 'Kepler-197',
       'Kepler-20', 'Kepler-208', 'Kepler-220', 'Kepler-251',
       'Kepler-265', 'Kepler-286', 'Kepler-299', 'Kepler-306',
       'Kepler-324', 'Kepler-341', 'Kepler-342', 'Kepler-444',
       'Kepler-62', 'Kepler-79', 'Kepler-82', 'V1298 Tau'], dtype='<U10')